In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
from tqdm import tqdm

In [ ]:
home_path = '../dataset/1_7/train'
# img_size = 544
img_w, img_h = 544, 544
details_level = 'all'
is_all = False
if details_level.upper() == 'ALL':
    is_all = True

print(f'Details all: {is_all}')

In [ ]:
files_list = os.listdir(home_path)
files_list.sort()
print(f'Total Image Files: {int(len(files_list) / 2)}')

In [ ]:
def cal_mode(X):
    X = X.flatten()
    idx, cnts = np.unique(X, return_counts=True)
    nid = np.argmax(cnts)
    return idx[nid]

def get_img_path(f_name):
    fn_list = f_name.split('.')
    img_name, ext = '.'.join(fn_list[:-1]), fn_list[-1]
    for extension in ['.jpg', '.png']:
        img_path = os.path.join(home_path, img_name+extension)
        if os.path.isfile(img_path):
            return img_path
        
        
def get_img_vals(pil_img=None):
    np_img = np.asarray(pil_img)
    
    raf = np_img[..., 0]
    gaf = np_img[..., 1]
    baf = np_img[..., 2]
    
    r_mode = cal_mode(raf)
    g_mode = cal_mode(gaf)
    b_mode = cal_mode(baf)
    
    #print(r_mode, g_mode, b_mode)
    return (r_mode, g_mode, b_mode)
    
    # plt.imshow(pil_img)
    # plt.show()

In [ ]:
def get_current_image(img_path):
    pil_img = Image.open(img_path)
    return pil_img
    
def get_obj_coordinates(curr_image, obj_data):
    c_img_w = curr_image.size[0]
    c_img_h = curr_image.size[1]
    
    cx = (float(data_item[1]) * c_img_w)
    cy = (float(data_item[2]) * c_img_h)
    bw = (float(data_item[3]) * c_img_w)
    bh = (float(data_item[4]) * c_img_h)
    # print(obj_data)
    
    left = int(cx - (bw / 2))
    top = int(cy - (bh / 2))
    right = int(cx + (bw / 2))
    bottom = int(cy + (bh / 2))
    
    cropped_img = curr_image.crop((left, top, right, bottom))
    return get_img_vals(cropped_img)

In [ ]:
columns_list = ['img_path',
               'img_size',
               'img_width',
               'img_height',
               
               'class_name',
               'class_idx',
               
               'ncx',
               'ncy',
               'nbw',
               'nbh']

In [ ]:
total_list = []

class_file = os.path.join(home_path, 'classes.txt')
with open(class_file) as fc:
    class_list = fc.readlines()

for file_name in tqdm(files_list):
    if file_name.endswith('.txt') and not file_name.startswith('class'):
        file_path = os.path.join(home_path, file_name)
        
        # if is_all
        if is_all:
            current_image = get_current_image(get_img_path(file_name))
            im_r_mode, im_g_mode, im_b_mode = get_img_vals(current_image)
        # print(im_r_mode, im_g_mode, im_b_mode)
        
        with open(file_path) as fp:
            # print(file_name)
            file_data = fp.readlines()
            for data_list in file_data:
                data_item = data_list.split(' ')
                if is_all:
                    obj_r_mode, obj_g_mode, obj_b_mode = get_obj_coordinates(curr_image=current_image,
                                                                             obj_data=data_item)
                try:
                    item_list = [
                        file_path,
                        
                        #img_size * img_size,
                        #img_size,
                        #img_size,
                        img_w * img_h,
                        img_w,
                        img_h,
                        
                        class_list[int(data_item[0])].replace('\n', ''),
                        int(data_item[0]),
                        
                        float(data_item[1]),
                        float(data_item[2]),
                        float(data_item[3]),
                        float(data_item[4]),
                    ]
                    
                    
                    if is_all:
                        item_list.extend([
                            int(im_r_mode),
                            int(im_g_mode),
                            int(im_b_mode),
                        
                            int(obj_r_mode),
                            int(obj_g_mode),
                            int(obj_b_mode),])
                        
                        
                    total_list.append(item_list)
                        
                        
                except IndexError:
                    print('\nClass names must match with # of objects')
                    print('Please check your object labels or total of class names.\n')
                    exit(0)


#print(columns_list)
#print(total_list)
if is_all:
    columns_list.extend(['im_r_mode', 'im_g_mode', 'im_b_mode',
                         'obj_r_mode', 'obj_g_mode','obj_b_mode',])
    
    
df = pd.DataFrame(data=total_list,
                  columns=columns_list)


#df['cx'], df['cy'], df['bw'], df['bh'] = df.ncx * img_size, df.ncy * img_size, df.nbw * img_size, df.nbh * img_size
df['cx'], df['cy'], df['bw'], df['bh'] = df.ncx * img_w, df.ncy * img_h, df.nbw * img_w, df.nbh * img_h

df['obj_px'] = df['bw'] * df['bh']
df['obj_img_ratio'] = df['obj_px'] / df['img_size']

In [ ]:
df.head()

In [ ]:
class_grp = df.groupby(['class_name'])
class_grp.mean()

In [ ]:
data_dict = dict()
for class_item in tqdm(class_grp):
    class_name = class_item[0]
    class_df = class_item[1]
    class_np = np.zeros((class_df.shape[0], 4, 4, 3))
    #print(class_df.shape[0])
    for idx, (r_idx, class_df_row) in enumerate(class_df.iterrows()):
        r_val = class_df_row['obj_r_mode']
        g_val = class_df_row['obj_g_mode']
        b_val = class_df_row['obj_b_mode']
        
        class_np[idx][..., 0].fill(r_val)
        class_np[idx][..., 1].fill(g_val)
        class_np[idx][..., 2].fill(b_val)
        
    data_dict.update({class_name: class_np.astype('uint8')})

In [ ]:
data_dict['Non'].shape

In [ ]:
plt.imshow(data_dict['Non'][9])

In [ ]:
def show_images(images, cols = 1, titles = None):
    """Display a list of images in a single figure with matplotlib.
    
    Parameters
    ---------
    images: List of np.arrays compatible with plt.imshow.
    
    cols (Default = 1): Number of columns in figure (number of rows is 
                        set to np.ceil(n_images/float(cols))).
    
    titles: List of titles corresponding to each image. Must have
            the same length as titles.
    """
    assert((titles is None)or (len(images) == len(titles)))
    n_images = len(images)
    if titles is None: titles = ['Image (%d)' % i for i in range(1,n_images + 1)]
    fig = plt.figure()
    for n, (image, title) in enumerate(zip(images, titles)):
        a = fig.add_subplot(np.ceil(n_images/float(cols)),cols, n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

In [ ]:
show_images(data_dict['Non'][:80], cols=20)

In [ ]:
def display_multiple_img(images, rows = 1, cols=1):
    figure, ax = plt.subplots(nrows=rows,ncols=cols )
    for ind,title in enumerate(images):
        ax.ravel()[ind].imshow(images[title])
        ax.ravel()[ind].set_title(title)
        ax.ravel()[ind].set_axis_off()
    plt.tight_layout()
    plt.show()
    
    
display_multiple_img(data_dict['Non'][:80], 2, 20)

# TODO BLOCK

Object-Wise Pixel-Level Info (Modes)

In [ ]:
pil_img = Image.open('../dataset/00897.jpg')
plt.imshow(pil_img)

In [ ]:
dfxi = df[df['img_path'] == '../dataset/00897.txt']
dfxi = dfxi[['cx', 'cy', 'bw', 'bh']].iloc[0]
dfxi

In [ ]:
left = int(dfxi.cx - (dfxi.bw / 2))
top = int(dfxi.cy - (dfxi.bh / 2))
right = int(dfxi.cx + (dfxi.bw / 2))
bottom = int(dfxi.cy + (dfxi.bh / 2))
left, top

In [ ]:
crop_img = pil_img.crop((left, top, right, bottom))
plt.imshow(crop_img)

In [ ]:
cropped_np_img = np.asarray(crop_img)
cropped_np_img.shape

In [ ]:
cal_mode(cropped_np_img[..., 0].flatten())

## Practice Block

In [ ]:
pil_img = Image.open('')
np_img = np.asarray(pil_img)
np_img.shape

In [ ]:
raf_img = np_img[..., 0]
gaf_img = np_img[..., 1]
baf_img = np_img[..., 2]

In [ ]:

idx, cnts

In [ ]:
np.argmax(cnts)

In [ ]:
idx[175]